In [1]:
import json
import os

In [2]:
dataset_name = "AR-LSAT"
split = "dev"
model_name = "gpt-4"
backup = "random"

program_path = os.path.join("outputs/logic_programs", f"{dataset_name}_{split}_{model_name}.json")
result_path = os.path.join("outputs/logic_inference", f"{dataset_name}_{split}_{model_name}_backup-{backup}.json")

with open(program_path, "r") as json_file:
    programs = json.load(json_file)

with open(result_path, "r") as json_file:
    results = json.load(json_file)

In [62]:
# if result["flag"] == "parsing error":
#     continue
# if result["flag"] == "execution error":
#     continue
i = 66
assert results[i]["id"] == programs[i]["id"]
# print(programs[i]["context"])
# print(programs[i]["options"])
print(results[i]["flag"])
print(results[i]["predicted_answer"])
print(programs[i]["raw_logic_programs"][0])
# for line in programs[i]["raw_logic_programs"]:
#     print(line)
print()
print("answer", programs[i]["answer"])

parsing error
C
Predicates:
University(x) ::: x is a university.
Private(x) ::: x is a private university.
IvyLeague(x) ::: x is an Ivy League university.
Research(x) ::: x is a research university.
Move(x, y) ::: x moved to y.
NewHaven(x) ::: x is located in New Haven.
Endowment(x, y) ::: x's endowment is valued at y.
Organize(x, y) ::: x is organized into y.
College(x) ::: x is a college.
School(x) ::: x is a school.
Residential(x) ::: x is a residential college.
GraduateSchool(x) ::: x is the Yale Graduate School of Arts and Sciences.
ProfessionalSchool(x) ::: x is a professional school.
Premises:
University(yale) ∧ Private(yale) ∧ IvyLeague(yale) ∧ Research(yale) ::: Yale University is a private Ivy League research university.
Move(yale, newHaven) ∧ Move(yale, year1716) ::: Yale University moved to New Haven in 1716.
Endowment(yale, 42.3billion) ::: Yale university's endowment was valued at $42.3 billion.
Organize(yale, {residentialColleges, graduateSchool, professionalSchools}) ∧ 

In [7]:
for program, result in zip(programs, results):
    assert result["id"] == program["id"]
    # if result["flag"] not "parsing error":
    #     continue
    if result["flag"] == "execution error":
        continue
    # print(program["context"])
    # print(program["options"])
    print(result["flag"])
    print(program["raw_logic_programs"][0])
    print()
    print("predicted_answer", result["predicted_answer"])
    print("answer", program["answer"])


parsing error
# Declarations
children = EnumSort([Fred, Juan, Marc, Paul, Nita, Rachel, Trisha])
lockers = EnumSort([1, 2, 3, 4, 5])
assigned = Function([children] -> [lockers])

# Constraints
ForAll([l:lockers], Or(Count([c:children], assigned(c) == l) == 1, And(Count([c:children], assigned(c) == l) == 2, Exists([b:boys, g:girls], And(assigned(b) == l, assigned(g) == l))))) ::: Each locker must be assigned to either one or two children, and each child must be assigned to exactly one locker
Exists([c:children], assigned(Juan) == assigned(c)) ::: Juan must share a locker
ForAll([c:children], assigned(Rachel) != assigned(c)) ::: Rachel cannot share a locker
ForAll([l:lockers], Implies(assigned(Nita) == l, And(assigned(Trisha) != l, assigned(Trisha) != l+1, assigned(Trisha) != l-1))) ::: Nita's locker cannot be adjacent to Trisha's locker
assigned(Fred) == 3 ::: Fred must be assigned to locker 3

# Options
Question ::: Which one of the following is a complete and accurate list of the chil

In [51]:
constants = {'Yale', '27'}
r'|'.join(list(constants))

'Yale|27'

In [45]:
{'Yale', '27'}

'Million|34'